In [0]:
# !pip install tensorflow==2.0.0-beta0

import tensorflow as tf
print(tf.__version__)

2.0.0-beta0


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['I love My dog',
             'I love you',
             'l love my Cat',
             'You all love my dog!!']

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

print(word_index)

{'love': 1, 'my': 2, 'i': 3, 'dog': 4, 'you': 5, 'l': 6, 'cat': 7, 'all': 8}


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['I love My dog',
             'I love you',
             'l love my Cat',
             'You all love my dog!!']

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

print(word_index)
print(sequences)

{'love': 1, 'my': 2, 'i': 3, 'dog': 4, 'you': 5, 'l': 6, 'cat': 7, 'all': 8}
[[3, 1, 2, 4], [3, 1, 5], [6, 1, 2, 7], [5, 8, 1, 2, 4]]


## NLP ON IMDB Dataset Using Deep Learning

In [0]:
# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed

# tf.enable_eager_execution()

# !pip install -q tensorflow-datasets

In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
len(training_labels_final)

25000

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b oh yeah jenna jameson did it again yeah baby this movie rocks it was one of the 1st movies i saw of her and i have to say i feel in love with her she was great in this move br br her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing br br i hope this comment helps and u can buy the movie the storyline is awesome is very unique and i'm sure u are going to like it jenna amazed us once more and no wonder the movie won so many
b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 258us/sample - loss: 0.5597 - accuracy: 0.7258 - val_loss: 0.4428 - val_accuracy: 0.8368
Epoch 2/10
25000/25000 [==============================] - 6s 254us/sample - loss: 0.3363 - accuracy: 0.8954 - val_loss: 0.4111 - val_accuracy: 0.8350
Epoch 3/10
25000/25000 [==============================] - 6s 253us/sample - loss: 0.1863 - accuracy: 0.9603 - val_loss: 0.4581 - val_accuracy: 0.8264
Epoch 4/10
25000/25000 [==============================] - 6s 227us/sample - loss: 0.1118 - accuracy: 0.9811 - val_loss: 0.4743 - val_accuracy: 0.8260
Epoch 5/10
25000/25000 [==============================] - 6s 224us/sample - loss: 0.0823 - accuracy: 0.9857 - val_loss: 0.5689 - val_accuracy: 0.8172
Epoch 6/10
25000/25000 [==============================] - 6s 230us/sample - loss: 0.0680 - accuracy: 0.9877 - val_loss: 0.5608 - val_accuracy: 0.8250
Epoch 7/10
25000/25000 [==========================

In [0]:
num_epochs = 10
model2.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 259us/sample - loss: 0.5728 - accuracy: 0.7226 - val_loss: 0.4079 - val_accuracy: 0.8372
Epoch 2/10
25000/25000 [==============================] - 6s 244us/sample - loss: 0.3464 - accuracy: 0.8580 - val_loss: 0.3364 - val_accuracy: 0.8573
Epoch 3/10
25000/25000 [==============================] - 6s 229us/sample - loss: 0.2790 - accuracy: 0.8863 - val_loss: 0.3297 - val_accuracy: 0.8583
Epoch 4/10
25000/25000 [==============================] - 6s 224us/sample - loss: 0.2419 - accuracy: 0.9052 - val_loss: 0.3400 - val_accuracy: 0.8557
Epoch 5/10
25000/25000 [==============================] - 7s 264us/sample - loss: 0.2149 - accuracy: 0.9180 - val_loss: 0.3580 - val_accuracy: 0.8514
Epoch 6/10
25000/25000 [==============================] - 6s 240us/sample - loss: 0.1935 - accuracy: 0.9294 - val_loss: 0.3829 - val_accuracy: 0.8456
Epoch 7/10
25000/25000 [==========================

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

## 2. NLP ON Sarcasm Dataset Using Deep Learning